In [6]:
#!pip install PyMuPDF

In [7]:
import glob
import fitz  # PyMuPDF
import re
pdf_path_list=glob.glob("../data/*/*.pdf",
                        recursive=True)
len(pdf_path_list)

187412

In [8]:
#フリーズするpdf
black_list=[
"2532737",
"5354879",
"4067971",
"4879242",
"1731857",
"7578398",
"5387112",
]

In [9]:

save_path="output.jsonl"

#init
if False:
    with open("log.txt","w") as f:
        f.write("")
    with open(save_path,"w") as f:
        f.write("")


with open("log.txt") as f:
    done_list=f.readlines()

done_list=[i.strip() for i in done_list]
done_list[:3]

['6391196', '6391023', '6391106']

In [10]:
import json
from tqdm import tqdm

path=pdf_path_list[14]

for path in tqdm(pdf_path_list):
    filename=path.split("/")[-1].split(".")[0]
    with open("log.txt","a") as f:
        f.write(filename+"\n")

    if filename in done_list:
        continue
    if filename in black_list:
        continue
    try:
        doc = fitz.open(path)
        text_list=[]
        for page in doc:
            tmp_text = page.get_text("text")
            blocks = page.get_text("blocks")
            #for block in blocks[1:-1]:
            for block in blocks:
                #4つめのblockにテキスト｡ それ以外は座標?
                block_4 = block[4]
                block_4 = re.sub('\n', '', block_4)
                block_4 = re.sub('\u3000', '', block_4)
                if block_4.find("<image:")==0:
                    continue
                if len(block_4) > 5:
                    text_list.append(block_4)

        txt="\n".join(text_list)
        if len(txt)==0:
            continue
        with open(save_path, "a", encoding='utf-8', errors='ignore') as f:
            f.write(json.dumps({"file":filename,"text":txt},ensure_ascii=False)+"\n")

    except:
        continue

100%|██████████| 187412/187412 [08:13<00:00, 379.48it/s]
